In [1]:
import ssl
import http.client
import json
import pandas as pd

class SofaScoreAPI:
    def __init__(self, match_id):
        self.match_id = match_id
        self.base_url = 'www.sofascore.com'

    def _make_request(self, endpoint):
        """
        Belirtilen endpoint'e HTTP isteği yapar ve yanıtı döner.
        """
        context = ssl._create_unverified_context()
        con = http.client.HTTPSConnection(self.base_url, context=context)
        try:
            con.request("GET", endpoint)
            response = con.getresponse()
            data = response.read()
            data = json.loads(data)
            return {"match_id":self.match_id,"response_code": response.status, "data": data}
        except Exception as e:
            return {"match_id":self.match_id,"response_code": response.status , "data": None }

    def get_lineups(self):
        """
        Kadro bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/lineups'
        return self._make_request(endpoint)

    def get_incidents(self):
        """
        Maç olaylarını döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/incidents'
        return self._make_request(endpoint)

    def get_graph(self):
        """
        Maç baskı grafiğini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/graph'
        return self._make_request(endpoint)

    def get_match_info(self):
        """
        Maç bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}'
        return self._make_request(endpoint)

    def get_odds(self):
        """
        Maç oran bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/odds/672/featured'
        return self._make_request(endpoint)


    def get_statistics(self):
        """
        Maç istatistiklerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/statistics'
        return self._make_request(endpoint)

    def fetch_all_data(self):
        """
        Tüm API endpoint'lerinden verileri toplar ve bir dict olarak döner.
        """
        return {
            "match_id": self.match_id,
            "lineups": self.get_lineups(),
            "incidents": self.get_incidents(),
            "graph": self.get_graph(),
            "match_info": self.get_match_info(),
            "odds": self.get_odds(),
            "pregame_form": self.get_pregame_form(),
            "statistics": self.get_statistics(),
        }

# Örnek kullanım:
match_id =12499750
sofa_api = SofaScoreAPI(match_id)
result_all = sofa_api.get_statistics()
result_all

{'match_id': 12499750,
 'response_code': 200,
 'data': {'statistics': [{'period': 'ALL',
    'groups': [{'groupName': 'Match overview',
      'statisticsItems': [{'name': 'Ball possession',
        'home': '56%',
        'away': '44%',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 56,
        'awayValue': 44,
        'renderType': 2,
        'key': 'ballPossession'},
       {'name': 'Expected goals',
        'home': '2.21',
        'away': '0.48',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 2.21,
        'awayValue': 0.48,
        'renderType': 1,
        'key': 'expectedGoals'},
       {'name': 'Big chances',
        'home': '6',
        'away': '0',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 6,
        'awayValue': 0,
        'renderType': 1,
        'key': 'bigChanceCreated'

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [5]:
from analiztest.models import Lineups, Incidents, Graph, MatchInfo, Odds, Statistics
# Örnek kullanım:
match_id = 13040469
sofa_api = SofaScoreAPI(match_id)

result_lineups = sofa_api.get_lineups()
result_incidents = sofa_api.get_incidents()
result_graph = sofa_api.get_graph()
result_match_info = sofa_api.get_match_info()
result_odds = sofa_api.get_odds()
result_statistics = sofa_api.get_statistics()

# Kaydetme işlemi
if result_lineups['data']:
    Lineups.objects.update_or_create(
        match_id=result_lineups['match_id'],
        defaults={'data': result_lineups['data']}
    )

if result_incidents['data']:
    Incidents.objects.update_or_create(
        match_id=result_incidents['match_id'],
        defaults={'data': result_incidents['data']}
    )

if result_graph['data']:
    Graph.objects.update_or_create(
        match_id=result_graph['match_id'],
        defaults={'data': result_graph['data']}
    )

if result_match_info['data']:
    MatchInfo.objects.update_or_create(
        match_id=result_match_info['match_id'],
        defaults={'data': result_match_info['data']}
    )

if result_odds['data']:
    Odds.objects.update_or_create(
        match_id=result_odds['match_id'],
        defaults={'data': result_odds['data']}
    )

if result_statistics['data']:
    Statistics.objects.update_or_create(
        match_id=result_statistics['match_id'],
        defaults={'data': result_statistics['data']}
    )
print("işlem bitti")

işlem bitti
